<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**Вывод**

Были загружены и обработаны данные. Пропуски в данных отсутствуют. Дубликаты были оставлены, так как недостаточно данных для подтверждения того, что это дубликаты. 

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Если домножить матрицу на обратимую (квадратную), то качество модели и предсказания не изменится. Изменятся лишь веса обучаемой модели.

**Обоснование:** 

$$
w' = ((XP)^T XP)^{-1} (XP)^T y,
$$
 
где P - обратимая матрица, X - исходная матрица признаков, w' - новые веса в закодированной матрице.

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^TX P)^{-1} P^TX^T y
$$

$$
w' = (P^T X^TX P)^{-1} P^TX^T y = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^TX^T y
$$

$$
(P^T)^{-1} P^T = E
$$

$$
w' = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^TX^T y = P^{-1} ((X^TX)^{-1} X^T y)
$$

$$
w' = P^{-1} w 
$$

## Алгоритм преобразования

**Алгоритм**

1) Обучить модель на исходных данных

2) Домножить признаки на квадратную матрицу и обучить модель на этих данных

3) Проверить совпадений результатов двух моделей

**Обоснование**

1) Для данных без преобразования

$$
a = Xw
$$

2) Для данных с преобразованием

$$
a' = (XP) w'
$$

$$
a' = (XP) w' = (XP) P^{-1} w = Xw
$$

## Проверка алгоритма

In [37]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [38]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.8, random_state=12345)

**Обучение на данных без кодировки**

In [39]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
r2 = r2_score(target_test, predictions)

**Кодирование исходных данных**

Всего признака 4, поэтому для кодирования необходимо домножить исходные данные на квадратную матрицу размерностью 4.

In [40]:
matrix = np.array([
    [1, 2, 3, 4],
    [4, 6, 7, 9],
    [-1, 4, 8, 10],
    [0, 7, 15, 3]
])

features_train_code = np.array(features_train) @ matrix
features_test_code = np.array(features_test) @ matrix

In [41]:
model = LinearRegression()
model.fit(features_train_code, target_train)
predictions = model.predict(features_test_code)
r2_code = r2_score(target_test, predictions)

**Проверка качества моделей**

In [42]:
print('Качество модели на исходных данных:', r2)
print('Качество модели на закодированных данных:',r2_code)

Качество модели на исходных данных: 0.42016377665505733
Качество модели на закодированных данных: 0.42016377665464144


**Восстановление исходной матрицы**

In [43]:
features_train_after_code = features_train_code @ np.linalg.inv(matrix)

In [44]:
features_train_after_code

array([[ 1.47922785e-11,  3.20000000e+01,  2.12000000e+04,
         1.00000000e+00],
       [ 1.00000000e+00,  2.00000000e+01,  3.64000000e+04,
         2.00000000e+00],
       [-3.44826390e-11,  4.80000000e+01,  2.82000000e+04,
        -3.11040083e-12],
       ...,
       [ 1.00000000e+00,  4.10000000e+01,  4.47000000e+04,
         1.00000000e+00],
       [ 5.29130073e-11,  2.20000000e+01,  5.01000000e+04,
         4.00000000e+00],
       [ 3.04553049e-11,  1.90000000e+01,  4.08000000e+04,
        -5.43656786e-12]])

In [46]:
np.array(features_train)

array([[0.00e+00, 3.20e+01, 2.12e+04, 1.00e+00],
       [1.00e+00, 2.00e+01, 3.64e+04, 2.00e+00],
       [0.00e+00, 4.80e+01, 2.82e+04, 0.00e+00],
       ...,
       [1.00e+00, 4.10e+01, 4.47e+04, 1.00e+00],
       [0.00e+00, 2.20e+01, 5.01e+04, 4.00e+00],
       [0.00e+00, 1.90e+01, 4.08e+04, 0.00e+00]])

**Вывод**

Было обучено две модели. По результатам обучения можно сделать вывод о том, что при кодировании качество модели и результат работы модели не изменяется.

Поэтому для различных компаний, которые работают с секретными данными клиентов возможно применение способа кодирования аданных при помощи умножения исходных данных на квадратную матрицу.

Также следует отметить, что данные восстанавливаются после кодирования, то есть потеряные они не будут.